# TFG

Primero importamos las librerías

In [2]:
import pandas as pd
import numpy as np

Importamos los datos, e imprimimos los primeros para comprobar que se han importado correctamente.

In [5]:
df = pd.read_csv('SolUsd.csv')
df.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2020-04-10,0.832005,1.313487,0.694187,0.951054,0.951054,87364276
1,2020-04-11,0.951054,1.049073,0.765020,0.776819,0.776819,43862444
2,2020-04-12,0.785448,0.956670,0.762426,0.882507,0.882507,38736897
3,2020-04-13,0.890760,0.891603,0.773976,0.777832,0.777832,18211285
4,2020-04-14,0.777832,0.796472,0.628169,0.661925,0.661925,16747614


Imprimimos la información de cada característica

In [7]:
df.tail()

,Date,Open,High,Low,Close,Adj Close,Volume
710,2022-03-21,88.605003,91.011986,87.207642,88.757202,88.757202,1598537312
711,2022-03-22,88.741905,93.766808,88.182777,90.512970,90.512970,1879747152
712,2022-03-23,90.513931,96.051842,89.811462,95.102684,95.102684,2099467745
713,2022-03-24,95.020905,103.455795,94.065506,102.223694,102.223694,3601646600
714,2022-03-25,102.301895,104.529800,99.998749,100.423592,100.423592,2283853056


In [6]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 715 entries, 0 to 714
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Date       715 non-null    object 
 1   Open       715 non-null    float64
 2   High       715 non-null    float64
 3   Low        715 non-null    float64
 4   Close      715 non-null    float64
 5   Adj Close  715 non-null    float64
 6   Volume     715 non-null    int64  
dtypes: float64(5), int64(1), object(1)
memory usage: 39.2+ KB
None
